In [106]:
import os
import requests
import json
import numpy as np
import tensorflow as tf
import pandas as pd
from config import api_key
from pprint import pprint
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense
from tensorflow.keras.datasets import mnist
from sklearn.metrics import classification_report

In [107]:
url = "https://play.limitlesstcg.com/api"
api_key = "key="+ api_key
query = "game=PTCG&format=standard&"
tournamentid = ""

In [108]:
response = requests.get(url+"/tournaments/"+tournamentid+"?"+query+api_key)

In [109]:
tournaments = response.json()
tournaments

[{'game': 'PTCG',
  'name': 'Late Night #103 on PTCG Live | Road to Japan',
  'date': '2023-05-30T16:00:00.000Z',
  'format': 'STANDARD',
  'id': '6406779924ea2b561a98792c',
  'players': 177},
 {'game': 'PTCG',
  'name': 'Free entry ║ Deck Out Mondays ║ 200 codes',
  'date': '2023-05-30T00:00:00.000Z',
  'format': 'STANDARD',
  'id': '646bf66c9b9f5808497a8464',
  'players': 92},
 {'game': 'PTCG',
  'name': "Sunny's Weekly #105",
  'date': '2023-05-29T22:05:00.000Z',
  'format': 'STANDARD',
  'id': '646c22f09b9f5808497a8618',
  'players': 60},
 {'game': 'PTCG',
  'name': 'FalaMeowth! "5/0 Hero" #11',
  'date': '2023-05-29T22:00:00.000Z',
  'format': 'STANDARD',
  'id': '64746f199b9f5808497aace0',
  'players': 22},
 {'game': 'PTCG',
  'name': 'Standard Pumpka Weekly',
  'date': '2023-05-29T18:00:00.000Z',
  'format': 'STANDARD',
  'id': '6473dad59b9f5808497aab90',
  'players': 64},
 {'game': 'PTCG',
  'name': 'Torédo Cup May #4',
  'date': '2023-05-29T15:30:00.000Z',
  'format': 'STANDAR

In [110]:
#setting up empty dataframe with all cards

# sets= [("BST"),("CRE"),("EVS"),("FST"),("BRS"),("ASR"),("LOR"),("SIT"),("SVI"),("PAL"),("OBF"),("SHF"),("CEL"),("PGO"),("CRZ"),("SSP"),("SVP"),("MCD22"),("MCD21")]
cards= []

# for set in sets:
#     for i in range (301):
#         cards.append(set + str(i))
# energytypes = ['energyG', 'energyF', 'energyL', 'energyP', 'energyM', 'energyW', 'energyD', 'energyR', 'energyY']

# for type in energytypes:
#     cards.append(type)
cards.append("archetype")

carddf = pd.DataFrame([], columns=[cards])
carddf["archetype"].astype(str)
carddf.head()

,archetype


In [111]:
tournament_names = []
decklists = []
deckarch = []

card_archetypes = []

for tournament in tournaments:
    tournament_names.append(tournament["name"])
    tournament_ID = tournament["id"]
    print(url+"/tournaments/"+tournament_ID+"/standings?"+query+api_key)
    response = requests.get(url+"/tournaments/"+tournament_ID+"/standings?"+query+api_key)
    data=response.json()
    data
    print(tournament["name"])
    for player in data:
        if player["decklist"] is not None:
            if player["deck"]["id"] != "other":
                idx = carddf.index.tolist() + ['end']
                carddf = carddf.reindex(idx, fill_value=0).reset_index(drop=True)
                if "pokemon" in player["decklist"]:
                    for card in player["decklist"]["pokemon"]:
                        poke_cardnumber=card["set"]+card["number"]
                        carddf.loc[len(carddf.index)-1, poke_cardnumber] = card["count"]
                if "trainer" in player["decklist"]:
                    for card in player["decklist"]["trainer"]:
                        poke_cardnumber=card["set"]+card["number"]
                        carddf.loc[len(carddf.index)-1, poke_cardnumber] = card["count"]      
                
                if "energy" in player["decklist"]:
                    for card in player["decklist"]["energy"]:
                        if card["number"].isnumeric():
                            poke_cardnumber=card["set"]+card["number"]
                            carddf.loc[len(carddf.index)-1, poke_cardnumber] = card["count"]    
                        else:  
                            poke_cardnumber="energy"+card["number"]
                            carddf.loc[len(carddf.index)-1, poke_cardnumber] = card["count"]
                carddf.loc[len(carddf.index)-1, "archetype"] = player["deck"]["id"]
                if player["deck"]["id"] not in card_archetypes:
                    card_archetypes.append(player["deck"]["id"])
carddf=carddf.fillna(0)


https://play.limitlesstcg.com/api/tournaments/6406779924ea2b561a98792c/standings?game=PTCG&format=standard&key=5ff01485ba33aa465c6d4bdc23d94e81
Late Night #103 on PTCG Live | Road to Japan
https://play.limitlesstcg.com/api/tournaments/646bf66c9b9f5808497a8464/standings?game=PTCG&format=standard&key=5ff01485ba33aa465c6d4bdc23d94e81
Free entry ║ Deck Out Mondays ║ 200 codes
https://play.limitlesstcg.com/api/tournaments/646c22f09b9f5808497a8618/standings?game=PTCG&format=standard&key=5ff01485ba33aa465c6d4bdc23d94e81
Sunny's Weekly #105
https://play.limitlesstcg.com/api/tournaments/64746f199b9f5808497aace0/standings?game=PTCG&format=standard&key=5ff01485ba33aa465c6d4bdc23d94e81
FalaMeowth! "5/0 Hero" #11
https://play.limitlesstcg.com/api/tournaments/6473dad59b9f5808497aab90/standings?game=PTCG&format=standard&key=5ff01485ba33aa465c6d4bdc23d94e81
Standard Pumpka Weekly
https://play.limitlesstcg.com/api/tournaments/6457b0ae9b9f5808497a2923/standings?game=PTCG&format=standard&key=5ff01485ba33

In [112]:
display(carddf.head())

display(carddf.tail())

,archetype,ASR60,SIT67,SIT68,SVI86,CRE61,CEL16,LOR74,CEL11,ASR46,...,LOR36,BST33,LOR37,CRZ29,LOR34,LOR40,SVI16,EVS66,BST9,SIT54
0,gardevoir-ex-sv,3.0,1.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,lugia-archeops,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,giratina-lz-box,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,gardevoir-ex-sv,3.0,1.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,gardevoir-ex-sv,3.0,1.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,archetype,ASR60,SIT67,SIT68,SVI86,CRE61,CEL16,LOR74,CEL11,ASR46,...,LOR36,BST33,LOR37,CRZ29,LOR34,LOR40,SVI16,EVS66,BST9,SIT54
2145,inteleon-urshifu,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2146,goodra-lz-box,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2147,gardevoir-ex-sv,0.0,4.0,4.0,2.0,1.0,1.0,0.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2148,hisuian-zoroark-vstar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2149,lost-zone-box,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
# print(carddf)
print(carddf.sum(axis=1, numeric_only=True).sum())

print(carddf["archetype"])


129000.0
                  archetype
0           gardevoir-ex-sv
1            lugia-archeops
2           giratina-lz-box
3           gardevoir-ex-sv
4           gardevoir-ex-sv
...                     ...
2145       inteleon-urshifu
2146          goodra-lz-box
2147        gardevoir-ex-sv
2148  hisuian-zoroark-vstar
2149          lost-zone-box

[2150 rows x 1 columns]


In [114]:
target = carddf["archetype"]
target_names = card_archetypes

In [115]:
# Get the features. 
data = carddf.drop("archetype", axis=1)
feature_names = data.columns
data.head()

,ASR60,SIT67,SIT68,SVI86,CRE61,CEL16,LOR74,CEL11,ASR46,BRS41,...,LOR36,BST33,LOR37,CRZ29,LOR34,LOR40,SVI16,EVS66,BST9,SIT54
0,3.0,1.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,1.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,1.0,4.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [117]:
# Support vector machine linear classifier
from sklearn.svm import SVC 
model = SVC(kernel='linear')
model.fit(X_train, y_train)

c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(kernel='linear')

In [118]:
# Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.987


In [119]:
target_names

['gardevoir-ex-sv',
 'lugia-archeops',
 'giratina-lz-box',
 'mew-genesect',
 'arceus-giratina',
 'lost-zone-box',
 'mew-meloetta',
 'gardevoir-mewtwo',
 'inteleon-urshifu',
 'hisuian-zoroark-vstar',
 'goodra-lz-box',
 'arceus-duraludon',
 'spidops-ex',
 'lzb-charizard',
 'arceus-vstar',
 'miraidon-flaaffy',
 'arcanine-ex-sv',
 'snorlax-stall',
 'palkia-vstar',
 'lunarock-pgo',
 'rotom-v',
 'primeape-bst',
 'miraidon-regieleki',
 'kyurem-vmax',
 'dondozo-svi',
 'flareon-vmax',
 'zeraora-v',
 'palkia-gardevoir',
 'dialga-magnezone',
 'oinkologne-ex',
 'arceus-ice-rider-calyrex',
 'miraidon-ex',
 'dakrai-vstar',
 'zoroark-evs',
 'rapid-strike-urshifu-vmax',
 'ice-rider-calyrex-palkia',
 'arceus-flying-pikachu',
 'arceus-goodra',
 'banette-ex-sv',
 'arceus-regidrago',
 'arceus-tapu-koko',
 'shadow-rider-calyrex-vmax',
 'hisuian-goodra-vstar',
 'crabominable-v',
 'arceus-mewtwo',
 'regidrago-vstar',
 'flaaffy-raichu',
 'arceus-rayquaza',
 'sandaconda-bst',
 'arceus-vulpix',
 'kyurem-palkia'

In [120]:
# Calculate the classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            labels=target_names))

                           precision    recall  f1-score   support

          gardevoir-ex-sv       0.98      1.00      0.99        51
           lugia-archeops       1.00      1.00      1.00        50
          giratina-lz-box       1.00      1.00      1.00        28
             mew-genesect       1.00      1.00      1.00        29
          arceus-giratina       1.00      1.00      1.00        37
            lost-zone-box       0.98      1.00      0.99       102
             mew-meloetta       1.00      1.00      1.00        27
         gardevoir-mewtwo       1.00      1.00      1.00         9
         inteleon-urshifu       1.00      1.00      1.00        15
    hisuian-zoroark-vstar       1.00      1.00      1.00         6
            goodra-lz-box       1.00      1.00      1.00        18
         arceus-duraludon       1.00      1.00      1.00        27
               spidops-ex       0.00      0.00      0.00         0
            lzb-charizard       1.00      1.00      1.00     

c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\d_hin\anaconda3\lib\site-packages\sklearn\metrics\_cla

In [ ]:
testdeck=
Test ={decklist,deck}

